In [1]:
import graphlab

In [2]:
image_train = graphlab.SFrame("/home/aunja/Downloads/image_train_data.csv")

This non-commercial license of GraphLab Create for academic use is assigned to anujaraj10@gmail.com and will expire on November 14, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1511940857.log


Finished parsing file /home/aunja/Downloads/image_train_data.csv

Parsing completed. Parsed 100 lines in 2.57709 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,str,array,array]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 1943 lines. Lines per second: 617.413

Finished parsing file /home/aunja/Downloads/image_train_data.csv

Parsing completed. Parsed 2005 lines in 3.2076 secs.

In [3]:
image_test =graphlab.SFrame("/home/aunja/Downloads/image_test_data.csv")

Finished parsing file /home/aunja/Downloads/image_test_data.csv

Parsing completed. Parsed 100 lines in 2.63221 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,str,array,array]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 1940 lines. Lines per second: 540.672

Finished parsing file /home/aunja/Downloads/image_test_data.csv

Parsing completed. Parsed 4000 lines in 5.06477 secs.

# # Exploring the image data

In [7]:
graphlab.canvas.set_target('ipynb')

In [8]:
image_train['image'].show()

# Train a classifier on the raw image pixels
We first start by training a classifier on just the raw pixels of the image.

In [9]:
raw_pixel_model = graphlab.logistic_classifier.create(image_train,target='label',
                                              features=['image_array'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 1908

...

| 9         | 17       | 1.000000  | 14.775413    | 0.503669          | 0.567010            |

| 10        | 18       | 1.000000  | 15.846547    | 0.506813          | 0.567010            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

# Make a prediction with the simple model based on raw pixels


In [10]:
image_test[0:3]['image'].show()

In [11]:
image_test[0:3]['label']

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

In [12]:
raw_pixel_model.predict(image_test[0:3])

dtype: str
Rows: 3
['bird', 'cat', 'bird']

The model makes wrong predictions for all three images.
# Evaluating raw pixel model on test data

In [13]:
raw_pixel_model.evaluate(image_test)

{'accuracy': 0.47325, 'auc': 0.7162946666666656, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |  automobile  |       bird      |  110  |
 |     bird     |       bird      |  510  |
 |     bird     |       dog       |  172  |
 |     bird     |       cat       |  183  |
 |     cat      |       dog       |  277  |
 |     dog      |       dog       |  394  |
 |     dog      |    automobile   |  114  |
 |     cat      |       cat       |  358  |
 |     cat      |    automobile   |  161  |
 |     dog      |       bird      |  234  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.47130839907266964, 'log_loss': 1.2191838050542225, 'precision': 

The accuracy of this model is poor, getting only about 46% accuracy.
# Can we improve the model using deep features
We only have 2005 data points, so it is not possible to train a deep neural network effectively with so little data. Instead, we will use transfer learning: using deep features trained on the full ImageNet dataset, we will train a simple model on this small dataset.

In [14]:
len(image_train)

2005

# Computing deep features for our images
The two lines below allow us to compute deep features. This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded.
(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

As we can see, the column deep_features already contains the pre-computed deep features for this data.


In [15]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242872, 1.09545, 0.0,0.39363, 0.0, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525088, 0.0, 0.0, 0.0,0.0, 0.0, 9.94829, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566016, 0.0, 0.0, 0.0,0.0, 0.0, 9.9972, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.1298, 0.0, 0.0,0.778194, 0.0, 0.758051, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71787, 0.0, 0.0, 0.0,0.0, 0.0, 9.33936, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57819, 0.0, 0.0, 0.0,0.0, 0.0, 9.00632, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0, 0.220678, 0.0,0.0, 0.0, 8.58053, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.237535, 0.0, 0.0,0.0, 0.0, 9.9908, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.57379, 0.0, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658936, 0.0, 0.0, 0.0,0.0, 0.0, 9.93748, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# Given the deep features, let's train a classifier

In [16]:
deep_features_model = graphlab.logistic_classifier.create(image_train,
                                                         features=['deep_features'],
                                                         target='label')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

WARNING: Detected extremely low variance for feature(s) 'deep_features' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

...

| 9         | 16       | 0.250000  | 19.832224    | 0.852895          | 0.806818            |

| 10        | 17       | 0.250000  | 21.367614    | 0.868023          | 0.806818            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

# Apply the deep features model to first few images of test set

In [17]:
image_test[0:3]['image'].show()

In [18]:
deep_features_model.predict(image_test[0:3])

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

The classifier with deep features gets all of these images right!

# Compute test_data accuracy of deep_features_model

As we can see, deep features provide us with significantly better accuracy (about 78%)

In [19]:
deep_features_model.evaluate(image_test)

{'accuracy': 0.785, 'auc': 0.9386901666666708, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |  automobile  |       cat       |   10  |
 |     dog      |       cat       |  197  |
 |  automobile  |       dog       |   5   |
 |     cat      |       bird      |   92  |
 |     bird     |       dog       |   52  |
 |     dog      |       bird      |   61  |
 |     cat      |    automobile   |   43  |
 |     bird     |       cat       |  103  |
 |     dog      |    automobile   |   19  |
 |     dog      |       dog       |  723  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.7833695563927372, 'log_loss': 0.636275275429469, 'precision': 0.78